In [1]:
import os
import magic
from secml.array import CArray
import numpy as np
import torch

from secml_malware.models.malconv import MalConv
from secml_malware.models.c_classifier_end2end_malware import CClassifierEnd2EndMalware, End2EndModel
#import secml_malware.models.c_classifier_pytorch

In [ ]:
net = MalConv()
net = CClassifierEnd2EndMalware(net)
#path = "secml_malware/data/trained/test_malconv.pth"
net.load_pretrained_model()

In [5]:
folder = "secml_malware/data/malware_samples/test_folder"

X = []
y = []
file_names = []
for i, f in enumerate(os.listdir(folder)):
    #print(f)
    path = os.path.join(folder, f)
    """
    if 'petya' not in path:
        continue
    if "PE32" not in magic.from_file(path):
        continue
    """
    with open(path, "rb") as file_handle:
        code = file_handle.read()
    x = End2EndModel.bytes_to_numpy(
        code, net.get_input_max_length(), 256, False
    )
    #y_train = np.ones((x.shape[0]))
    #print(x, y_train)
    #print(x.shape)

    X.append(x)
    y.append([1.0])
    file_names.append(path)

In [6]:
folder = "secml_malware/data/goodware_samples/my_samples"
for i, f in enumerate(os.listdir(folder)):
    #print(f)
    path = os.path.join(folder, f)
    """
    if 'petya' not in path:
        continue
    if "PE32" not in magic.from_file(path):
        continue
    """
    with open(path, "rb") as file_handle:
        code = file_handle.read()
    x = End2EndModel.bytes_to_numpy(
        code, net.get_input_max_length(), 256, False
    )
    #y_train = np.ones((x.shape[0]))
    #print(x, y_train)
    #print(x.shape)

    X.append(x)
    y.append([0.0])
    file_names.append(path)

In [7]:
print(len(X))
print(len(y))

20
20


In [8]:
input_X = CArray(X)
#input_y = CArray(y)
y = torch.FloatTensor(y)
#y = torch.unsqueeze(y, 1)
y = torch.reshape(y, (-1, ))
input_y = CArray(y)
print(input_y.shape)
print(input_y)

(20,)
CArray([1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.])


In [33]:
import copy
print(net)
net2 = copy.deepcopy(net)
net2.epochs = 2
print(net2)

CClassifierEnd2EndMalware{'classes': array([0, 1]), 'n_features': 1048576, 'preprocess': None, 'n_jobs': 1, 'model': MalConv(
  (embedding_1): Embedding(257, 8)
  (conv1d_1): Conv1d(8, 128, kernel_size=(500,), stride=(500,))
  (conv1d_2): Conv1d(8, 128, kernel_size=(500,), stride=(500,))
  (dense_1): Linear(in_features=128, out_features=128, bias=True)
  (dense_2): Linear(in_features=128, out_features=1, bias=True)
), 'trained': False, 'input_shape': (1, 1048576), 'softmax_outputs': False, 'batch_size': 256, 'loss': <function binary_cross_entropy at 0x7faa12db7af0>, 'optimizer': SGD (
Parameter Group 0
    dampening: 0
    lr: 0.01
    momentum: 0.9
    nesterov: True
    weight_decay: 0.001
), 'optimizer_scheduler': None, 'epochs': 100, 'plus_version': False, 'train_transform': Lambda()}
CClassifierEnd2EndMalware{'classes': array([0, 1]), 'n_features': 1048576, 'preprocess': None, 'n_jobs': 1, 'model': MalConv(
  (embedding_1): Embedding(257, 8)
  (conv1d_1): Conv1d(8, 128, kernel_siz

In [28]:
new_model = net2.fit(input_X, input_y)
print(net2)
net2._model = new_model
print(net2)

CClassifierEnd2EndMalware{'classes': CArray(2,)(dense: [0. 1.]), 'n_features': 1048576, 'preprocess': None, 'n_jobs': 1, 'model': MalConv(
  (embedding_1): Embedding(257, 8)
  (conv1d_1): Conv1d(8, 128, kernel_size=(500,), stride=(500,))
  (conv1d_2): Conv1d(8, 128, kernel_size=(500,), stride=(500,))
  (dense_1): Linear(in_features=128, out_features=128, bias=True)
  (dense_2): Linear(in_features=128, out_features=1, bias=True)
), 'trained': True, 'input_shape': (1, 1048576), 'softmax_outputs': False, 'batch_size': 256, 'loss': <function binary_cross_entropy at 0x7faa12db7af0>, 'optimizer': SGD (
Parameter Group 0
    dampening: 0
    lr: 0.01
    momentum: 0.9
    nesterov: True
    weight_decay: 0.001
), 'optimizer_scheduler': None, 'epochs': 2, 'plus_version': False, 'train_transform': Lambda()}
CClassifierEnd2EndMalware{'classes': CArray(2,)(dense: [0. 1.]), 'n_features': 1048576, 'preprocess': None, 'n_jobs': 1, 'model': MalConv(
  (embedding_1): Embedding(257, 8)
  (conv1d_1): Co

In [29]:
print(net2.predict(input_X))

CArray([1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0])


In [30]:
print(net.predict(input_X))

CArray([1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0])


In [31]:
torch.save(net2._model.state_dict(), 'secml_malware/data/trained/test_malconv.pth')

Here, I will try to divide the input into number of ablations and pass them to the 'vanilla' MalConv model

In [75]:
num_ablations = 4
max_input_size = 2**20
ablation_size = int(max_input_size/num_ablations)
padding_value = 256
new_X = []
new_Y = []
for i, x in enumerate(X):
    #print(i, x)
    splitted_x_array = np.split(x, num_ablations)
    for j, splitted_x in enumerate(splitted_x_array):
        byte_array = np.ones((max_input_size,), dtype=np.int16) * padding_value
        byte_array[j*ablation_size : (j+1)*ablation_size] = splitted_x
        new_X.append(byte_array)
    new_Y.append(np.ones((num_ablations)) if y[i]==1 else np.zeros((num_ablations)))
new_X = np.asarray(new_X)
#new_X = np.reshape(new_X, (-1, 1))
new_Y = np.asarray(new_Y)
print(new_X.shape, new_Y)

(80, 1048576) [[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [79]:
new_X = torch.FloatTensor(new_X)
#new_X = torch.reshape(new_X, (new_X.shape[0] * new_X.shape[1], new_X.shape[2]))
input_X = CArray(new_X)
print(input_X.shape)
new_Y = torch.FloatTensor(new_Y)
new_Y = torch.reshape(new_Y, (-1, 1))
input_Y = CArray(new_Y)
print(input_X, input_Y)

(80, 1048576)
CArray([[ 77.  90. 144. ... 256. 256. 256.]
 [256. 256. 256. ... 256. 256. 256.]
 [256. 256. 256. ... 256. 256. 256.]
 ...
 [256. 256. 256. ... 256. 256. 256.]
 [256. 256. 256. ... 256. 256. 256.]
 [256. 256. 256. ... 256. 256. 256.]]) CArray([[1.]
 [1.]
 [1.]
 ...
 [0.]
 [0.]
 [0.]])


In [73]:
y_pred = net2.predict(input_X)

In [81]:
from secml.ml.peval.metrics import CMetric
peval = CMetric.create('accuracy')
print(peval.performance_score(input_Y, y_pred))

0.5875


Here, I will try to create multiple MalConv models for different ablations

In [129]:
num_ablations = 5
max_input_size = 2**20-1
ablation_size = int(max_input_size/num_ablations)

In [130]:
net = MalConv(max_input_size = ablation_size)
net = CClassifierEnd2EndMalware(net, input_shape = (1, ablation_size))
net._n_features = ablation_size
#net.load_pretrained_model()

In [158]:
nets = []
for i in range(num_ablations):
    net = MalConv(max_input_size = ablation_size)
    net = CClassifierEnd2EndMalware(net, input_shape = (1, ablation_size))
    net._n_features = ablation_size
    nets.append(net)
print(nets)

[CClassifierEnd2EndMalware{'classes': None, 'n_features': 209715, 'preprocess': None, 'n_jobs': 1, 'model': MalConv(
  (embedding_1): Embedding(257, 8)
  (conv1d_1): Conv1d(8, 128, kernel_size=(500,), stride=(500,))
  (conv1d_2): Conv1d(8, 128, kernel_size=(500,), stride=(500,))
  (dense_1): Linear(in_features=128, out_features=128, bias=True)
  (dense_2): Linear(in_features=128, out_features=1, bias=True)
), 'trained': False, 'input_shape': (1, 209715), 'softmax_outputs': False, 'batch_size': 256, 'loss': <function binary_cross_entropy at 0x7faa12db7af0>, 'optimizer': SGD (
Parameter Group 0
    dampening: 0
    lr: 0.01
    momentum: 0.9
    nesterov: True
    weight_decay: 0.001
), 'optimizer_scheduler': None, 'epochs': 100, 'plus_version': False, 'train_transform': Lambda()}, CClassifierEnd2EndMalware{'classes': None, 'n_features': 209715, 'preprocess': None, 'n_jobs': 1, 'model': MalConv(
  (embedding_1): Embedding(257, 8)
  (conv1d_1): Conv1d(8, 128, kernel_size=(500,), stride=(5

In [185]:
new_X = []
new_Y = []
for i, x in enumerate(X):
    splitted_x_array = np.split(x[:-1], num_ablations)  ##here, -1 to make it divisible of 5
    new_X.append(splitted_x_array)
    new_Y.append(np.ones((num_ablations)) if y[i]==1 else np.zeros((num_ablations)))
new_X = np.asarray(new_X)
new_Y = np.asarray(new_Y)
print(new_X.shape)
print(new_Y.shape)

(20, 5, 209715)
(20, 5)


In [186]:
new_X = torch.FloatTensor(new_X)
#new_X = torch.reshape(new_X, (new_X.shape[0] * new_X.shape[1], new_X.shape[2]))
print(new_X.shape)
#input_X = CArray(new_X, copy=True, shape=(new_X.shape[0], new_X.shape[1], new_X.shape[2]), tosparse=False)
#print(input_X.shape)
new_Y = torch.FloatTensor(new_Y)
#new_Y = torch.reshape(new_Y, (-1, 1))
input_Y = CArray(new_Y)
print(input_X, input_Y)

torch.Size([20, 5, 209715])
CArray([[ 77.  90. 144. ... 256. 256. 256.]
 [ 77.  90. 144. ... 256. 256. 256.]
 [ 77.  90. 144. ... 256. 256. 256.]
 ...
 [ 77.  90. 144. ... 256. 256. 256.]
 [ 77.  90. 144. ... 256. 256. 256.]
 [ 77.  90. 144. ... 256. 256. 256.]]) CArray([[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 ...
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]])


In [219]:
for i in range(len(nets)):
    nets[i].epochs = 5
    nets[i]._model = nets[i].fit(new_X[:, i, :], new_Y[:, i])

In [224]:
y_preds = []
for i in range(len(nets)):
    y_pred = nets[i].predict(new_X[:, i, :])
    #print(y_pred)
    y_preds.append(y_pred.tondarray())
y_preds = np.asarray(y_preds)
print(y_preds)

[[1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 0 1 0 0]
 [1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 0]
 [1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 0]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1]]


In [221]:
from sklearn.metrics import accuracy_score
preds_acc_perc = []
for i in range(new_Y.shape[0]):
    preds_acc_perc.append(accuracy_score(new_Y[i, :], y_preds[:, i]))
preds_acc_perc = np.asarray(preds_acc_perc)
print(preds_acc_perc)
print("Accuracy without majority voting: ", np.mean(preds_acc_perc))

[1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  0.8 0.2 0.4 1.  0.4 0.2
 0.4 0.8]
Accuracy without majority voting:  0.8099999999999999


In [222]:
def get_majority_voting(y_detailed, sample_no):
    votes = []
    for i in range(sample_no):
        y_per_sample = y_detailed[:, i]
        mal_vote = np.count_nonzero(y_per_sample)
        ben_vote = np.count_nonzero(y_per_sample==0)
        if(mal_vote >= ben_vote): votes.append(1)
        else: votes.append(0)

    return votes

In [223]:
votes = get_majority_voting(y_preds, len(y))
print(votes)
print("Accuracy with majority voting: ", accuracy_score(y, votes))

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0]
Accuracy with majority voting:  0.75


Let's try to pass smaller input to the original MalConv model

In [225]:
net = MalConv()
net = CClassifierEnd2EndMalware(net)
net.load_pretrained_model()

In [226]:
print(net)

CClassifierEnd2EndMalware{'classes': array([0, 1]), 'n_features': 1048576, 'preprocess': None, 'n_jobs': 1, 'model': MalConv(
  (embedding_1): Embedding(257, 8)
  (conv1d_1): Conv1d(8, 128, kernel_size=(500,), stride=(500,))
  (conv1d_2): Conv1d(8, 128, kernel_size=(500,), stride=(500,))
  (dense_1): Linear(in_features=128, out_features=128, bias=True)
  (dense_2): Linear(in_features=128, out_features=1, bias=True)
), 'trained': False, 'input_shape': (1, 1048576), 'softmax_outputs': False, 'batch_size': 256, 'loss': <function binary_cross_entropy at 0x7faa12db7af0>, 'optimizer': SGD (
Parameter Group 0
    dampening: 0
    lr: 0.01
    momentum: 0.9
    nesterov: True
    weight_decay: 0.001
), 'optimizer_scheduler': None, 'epochs': 100, 'plus_version': False, 'train_transform': Lambda()}


In [2]:
from secml_malware.smoothed_malconv import get_dataset, create_smoothed_malconv, modify_dataset_for_smoothed_malconv, pad_ablated_input, train_model, model_predict, get_majority_voting

In [3]:
mal_path = "/Users/shoumik/Desktop/UMD 1st semester/Computer & Network Security/Project/Codes/MalConv-New/FullDataset/output/malware"
ben_path = "/Users/shoumik/Desktop/UMD 1st semester/Computer & Network Security/Project/Codes/MalConv-New/FullDataset/output/benign"
X, y, file_names = get_dataset(mal_path, ben_path, 2**16, 20)

In [19]:
print(X[0].shape)

(65536,)


In [10]:
net._epochs = 2
new_model = net._model

In [11]:
print(new_model)

MalConv(
  (embedding_1): Embedding(257, 8)
  (conv1d_1): Conv1d(8, 128, kernel_size=(500,), stride=(500,))
  (conv1d_2): Conv1d(8, 128, kernel_size=(500,), stride=(500,))
  (dense_1): Linear(in_features=128, out_features=128, bias=True)
  (dense_2): Linear(in_features=128, out_features=1, bias=True)
)


In [12]:
from torch.autograd import Variable
import torchvision
import torch.nn as nn

In [14]:
test_ipt = Variable(torch.zeros(1, 2**18))
#embedding = new_model.embedding_1(test_ipt)
embedding = net.embed(test_ipt)
conv1d_1 = new_model.conv1d_1(embedding)
conv1d_2 = new_model.conv1d_2(embedding)
conv1d_1_activation = torch.relu(conv1d_1)
conv1d_2_activation = torch.sigmoid(conv1d_2)
multiply_1 = conv1d_1_activation * conv1d_2_activation
global_max_pooling1d_1 = torch.nn.functional.max_pool1d(input=multiply_1, kernel_size=multiply_1.size()[2:])
global_max_pooling1d_1_flatten = global_max_pooling1d_1.view(global_max_pooling1d_1.size(0), -1)

print(multiply_1.shape)
print(global_max_pooling1d_1_flatten.size(1))
#print(global_max_pooling1d_1_flatten)

torch.Size([1, 128, 524])
128


In [255]:
vgg_loader = torchvision.models.vgg11
vgg = vgg_loader(pretrained=True)
print(vgg)

Downloading: "https://download.pytorch.org/models/vgg11-8a719046.pth" to /Users/shoumik/.cache/torch/hub/checkpoints/vgg11-8a719046.pth


  0%|          | 0.00/507M [00:00<?, ?B/s]

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace=True)
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
 

**Here, I will try to use the custom malconv model that has trainable fully connected layer**

In [3]:
from secml_malware.custom_malconv import Custom_MalConv

In [4]:
net = Custom_MalConv(max_input_size=2**20, unfreeze=True)
net = CClassifierEnd2EndMalware(net)
net._n_features = 2**20

In [5]:
print(net)

CClassifierEnd2EndMalware{'classes': None, 'n_features': 1048576, 'preprocess': None, 'n_jobs': 1, 'model': Custom_MalConv(
  (embedding_1): Embedding(257, 8)
  (conv1d_1): Conv1d(8, 128, kernel_size=(500,), stride=(500,))
  (conv1d_2): Conv1d(8, 128, kernel_size=(500,), stride=(500,))
  (classifier): Sequential(
    (0): Linear(in_features=128, out_features=128, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=128, out_features=1, bias=True)
    (3): Sigmoid()
  )
), 'trained': False, 'input_shape': (1, 1048576), 'softmax_outputs': False, 'batch_size': 256, 'loss': <function binary_cross_entropy at 0x7fba992d7af0>, 'optimizer': SGD (
Parameter Group 0
    dampening: 0
    lr: 0.01
    momentum: 0.9
    nesterov: True
    weight_decay: 0.001
), 'optimizer_scheduler': None, 'epochs': 100, 'plus_version': False, 'train_transform': Lambda()}


In [6]:
mal_path = "/Users/shoumik/Desktop/UMD 1st semester/Computer & Network Security/Project/Codes/MalConv-New/FullDataset/output/malware"
ben_path = "/Users/shoumik/Desktop/UMD 1st semester/Computer & Network Security/Project/Codes/MalConv-New/FullDataset/output/benign"
X, y, file_names = get_dataset(mal_path, ben_path, 2 ** 20, 20)
print(y)

[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]


In [7]:
new_X, new_y = modify_dataset_for_smoothed_malconv(X, np.reshape(y, (-1)), 4)
print(new_X.shape, new_y.shape)
print(type(new_X))
print(new_X)

(40, 4, 262144) (40, 4)
<class 'numpy.ndarray'>
[[[ 77.  90. 144. ... 256. 256. 256.]
  [256. 256. 256. ... 256. 256. 256.]
  [256. 256. 256. ... 256. 256. 256.]
  [256. 256. 256. ... 256. 256. 256.]]

 [[ 77.  90. 144. ... 160.  53. 250.]
  [204. 109.  69. ... 256. 256. 256.]
  [256. 256. 256. ... 256. 256. 256.]
  [256. 256. 256. ... 256. 256. 256.]]

 [[ 77.  90. 144. ... 256. 256. 256.]
  [256. 256. 256. ... 256. 256. 256.]
  [256. 256. 256. ... 256. 256. 256.]
  [256. 256. 256. ... 256. 256. 256.]]

 ...

 [[ 77.  90. 144. ... 256. 256. 256.]
  [256. 256. 256. ... 256. 256. 256.]
  [256. 256. 256. ... 256. 256. 256.]
  [256. 256. 256. ... 256. 256. 256.]]

 [[ 77.  90. 144. ... 235.  26.  80.]
  [232. 223.  60. ... 256. 256. 256.]
  [256. 256. 256. ... 256. 256. 256.]
  [256. 256. 256. ... 256. 256. 256.]]

 [[ 77.  90. 144. ... 256. 256. 256.]
  [256. 256. 256. ... 256. 256. 256.]
  [256. 256. 256. ... 256. 256. 256.]
  [256. 256. 256. ... 256. 256. 256.]]]


In [8]:
print(new_X[0, 0, :].shape)
new_padded_X = []
ablation_idx = 0
for i in range(new_X.shape[0]):
    temp_X = pad_ablated_input(new_X[i, ablation_idx, :], ablation_idx)
    new_padded_X.append(temp_X)
new_padded_X = np.asarray(new_padded_X)
print(new_padded_X.shape)
print(new_padded_X)

(262144,)
(40, 1048576)
[[ 77.  90. 144. ... 256. 256. 256.]
 [ 77.  90. 144. ... 256. 256. 256.]
 [ 77.  90. 144. ... 256. 256. 256.]
 ...
 [ 77.  90. 144. ... 256. 256. 256.]
 [ 77.  90. 144. ... 256. 256. 256.]
 [ 77.  90. 144. ... 256. 256. 256.]]


In [9]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(new_padded_X, new_y, test_size=0.40, random_state=1)
print(x_train.shape, x_test.shape)
print(y_train.shape)
print(y_test)

(24, 1048576) (16, 1048576)
(24, 4)
[[1. 1. 1. 1.]
 [0. 0. 0. 0.]
 [1. 1. 1. 1.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [1. 1. 1. 1.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [1. 1. 1. 1.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [ ]:
#out = net.embedd_and_forward(net.embed(torch.FloatTensor(X)))
#y = np.reshape(y, (-1))
print(new_y)
net._epochs = 2
out = net.fit(x_train, y_train[:, ablation_idx])
#out = net.fit(x_train[:, ablation_idx, :], y_train[:, ablation_idx])
print(out)
#net.fit(torch.FloatTensor(X))

[[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [ ]:
net._model = out
print(net)

In [ ]:
from sklearn.metrics import accuracy_score

preds = net.predict(CArray(x_train))
#preds = net.predict(CArray(x_train[:, ablation_idx, :]))
print(preds)
accuracy_score(preds.tondarray(), y_train[:, ablation_idx])

In [ ]:
preds = net.predict(CArray(x_test))
#preds = net.predict(CArray(x_test[:, ablation_idx, :]))
print(preds)
accuracy_score(preds.tondarray(), y_test[:, ablation_idx])